# Lab 6: SAR Data Processing

**Purpose:** The purpose of this lab is to explore processing SAR data on Earth Engine and become familiar with SAR data properties. RTC and speckle filtering preprocessing will be explored and an example of change detection.

In [ ]:
# import ee api and geemap package
import ee
import math
import geemap
import pandas as pd
from geemap import colormaps as cmaps

In [ ]:
# try to initalize an ee session
# if not authenticated then run auth workflow and initialize
try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

## Sentinel-1 Data

[Sentinel-1](https://earth.esa.int/web/sentinel/missions/sentinel-1) is a space mission funded by the European Union and carried out by the European Space Agency (ESA) within the Copernicus Programme. Sentinel-1 collects C-band synthetic aperture radar (SAR) imagery at a variety of polarizations and resolutions. Since radar data requires several specialized algorithms to obtain calibrated, orthorectified imagery, there are some description document describes pre-processing of Sentinel-1 data in Earth Engine.

## Earth Engine Preprocessing

Imagery in the Earth Engine 'COPERNICUS/S1_GRD' Sentinel-1 ImageCollection is consists of Level-1 Ground Range Detected (GRD) scenes processed to backscatter coefficient (σ°) in decibels (dB). The backscatter coefficient represents target backscattering area (radar cross-section) per unit ground area. Because it can vary by several orders of magnitude, it is converted to dB as $10\times log10\sigma^°$. It measures whether the radiated terrain scatters the incident microwave radiation preferentially away from the SAR sensor dB < 0) or towards the SAR sensor dB > 0). This scattering behavior depends on the physical characteristics of the terrain, primarily the geometry of the terrain elements and their electromagnetic characteristics.

Earth Engine uses the following preprocessing steps (as implemented by the Sentinel-1 Toolbox) to derive the backscatter coefficient in each pixel:

1. Apply orbit file:
Updates orbit metadata with a restituted orbit file (or a precise orbit file if the restituted one is not available).
2. GRD border noise removal:
Removes low intensity noise and invalid data on scene edges. (As of January 12, 2018)
3. Thermal noise removal:
Removes additive noise in sub-swaths to help reduce discontinuities between sub-swaths for scenes in multi-swath acquisition modes. (This operation cannot be applied to images produced before July 2015)
4. Radiometric calibration:
Computes backscatter intensity using sensor calibration parameters in the GRD metadata.
5. Geometric terrain correction (orthorectification):
Converts data from ground range geometry, which does not take terrain into account, to σ° using the SRTM 30 meter DEM or the ASTER DEM for high latitudes (greater than 60° or less than -60°).

## Metadata and Filtering

Sentinel-1 data is collected with several different instrument configurations, resolutions, band combinations during both ascending and descending orbits. Because of this heterogeneity, it's usually necessary to filter the data to down to a homogeneous subset before starting processing. This process is outlined below in the Metadata and Filtering section.


To create a homogeneous subset of Sentinel-1 data, it will usually be necessary to filter the collection using metadata properties. The common metadata fields used for filtering include these properties:

1. `transmitterReceiverPolarisation`: ['VV'], ['HH'], ['VV', 'VH'], or ['HH', 'HV']
2. `instrumentMode`: 'IW' (Interferometric Wide Swath), 'EW' (Extra Wide Swath) or 'SM' (Strip Map). See [this reference](https://sentinel.esa.int/web/sentinel/user-guides/sentinel-1-sar/acquisition-modes) for details.
3. `orbitProperties_pass`: 'ASCENDING' or 'DESCENDING'
4. `resolution_meters`: 10, 25 or 40
5. `resolution`: 'M' (medium) or 'H' (high). See [this reference](https://sentinel.esa.int/web/sentinel/user-guides/sentinel-1-sar/resolutions/level-1-ground-range-detected) for details.


The following code filters the Sentinel-1 collection by `transmitterReceiverPolarisation`, `instrumentMode`, and `orbitProperties_pass` properties, then calculates composites for several observation combinations that are displayed in the map to demonstrate how these characteristics affect the data.



In [ ]:
# Load the Sentinel-1 ImageCollection, filter to Jun-Sep 2020 observations.
sentinel1 = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filterDate('2020-06-01', '2020-10-01')
)

In [ ]:
# Filter the Sentinel-1 collection by metadata properties.
vv_vh_iw = (
    sentinel1
    # Filter to get images with VV and VH dual polarization.
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
    # Filter to get images collected in interferometric wide swath mode.
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
)


In [ ]:
# Separate ascending and descending orbit images into distinct collections.
vv_vh_iw_asc = vv_vh_iw.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
vv_vh_iw_desc = vv_vh_iw.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))

Calculate temporal means for various observations to use for visualization.


In [ ]:
# Mean VV ascending.
vv_iw_asc_mean = vv_vh_iw_asc.select('VV').mean();
# Mean VV descending.
vv_iw_desc_mean = vv_vh_iw_desc.select('VV').mean();
# Mean VV for combined ascending and descending image collections.
vv_iw_asc_desc_mean = vv_vh_iw.select('VV').mean();


In [ ]:
# Mean VH ascending.
vh_iw_asc_mean = vv_vh_iw_asc.select('VH').mean();
# Mean VH descending.
vh_iw_desc_mean = vv_vh_iw_desc.select('VH').mean();
# Mean VH for combined ascending and descending image collections.
vh_iw_asc_desc_mean = vv_vh_iw.select('VH').mean();

In [ ]:
# Visualize the results
Map = geemap.Map()

Map.setCenter(-73.8719, 4.512, 9);  # Bogota, Colombia

# Display the temporal means for various observations, compare them.
Map.addLayer(vv_iw_asc_mean, {"min": -25, "max": 0}, 'VV Asc Mean');
Map.addLayer(vv_iw_desc_mean , {"min": -25, "max": 0}, 'VV Desc Mean');
Map.addLayer(vv_iw_asc_desc_mean, {"min": -25, "max": 0}, 'VV Asc/Desc Mean');

Map.addLayerControl()

Map

## SAR Scale

SAR backscatter are recorded in both return strength and phase. This data is converted to ground range dectected (GRD) imagery which is the imagery we just looked at. These intensity images represent the absolute backscatter of the surface imaged. GRD images can be stored using several different scales, including **power**, **amplitude**, and **dB**. As mentioned before the default scale of Sentinel-1 data is dB. However, in some cases, it may be desirable to convert the actual pixel values to a different scale.



### Power Scale

The values in this scale are generally very close to zero, so the dynamic range of the SAR image can be easily skewed by a few bright scatterers in the image. Power scale is the most appropriate for statistical analysis of the SAR dataset, but may not always be the best option for data visualization.

Earth Engine stores Sentinel-1 data as power scale but it is not a searchable dataset: "COPERNICUS/S1_GRD_FLOAT"

When viewing a SAR image in power scale, it may appear mostly or all black, and you may need to adjust the stretch to see features in the image. Often applying a stretch of 2 standard deviations, or setting the Min-Max stretch values to 0 and 0.3, will greatly improve the appearance of the image.

In [ ]:
# read in the power scale imagecollection
s1_power_asc = (
    ee.ImageCollection("COPERNICUS/S1_GRD_FLOAT")
    .filterDate('2020-06-01', '2020-10-01')
    # Filter to get images with VV and VH dual polarization.
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
    # Filter to get images collected in interferometric wide swath mode.
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    # filter orbit pass
    .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
)

In [ ]:
# calculate power from the dB data
def db_to_power(image):
    return ee.Image.constant(10).pow(image.divide(10))


In [ ]:
# apply dB -> power scaling
power_from_db = vv_vh_iw_asc.map(db_to_power)

In [ ]:
# Visualize the results
Map = geemap.Map()

Map.setCenter(-73.8719, 4.512, 9);  # Bogota, Colombia

Map.addLayer(s1_power_asc.mean(), {"bands":"VV", "min": 0, "max": 0.3}, 'Original power image');
Map.addLayer(power_from_db.mean(), {"bands":"VV", "min": 0, "max": 0.3}, 'Power from dB image');

Map.addLayerControl()

Map

### Amplitude Scale

Amplitude scale is the square root of the power scale values. This brightens the darker pixels and darkens the brighter pixels, narrowing the dynamic range of the image. In many cases, amplitude scale presents a pleasing grayscale display of RTC images. Amplitude scale works well for calculating log difference ratios (see ASF Sentinel-1 RTC Product Guide).

In [ ]:
# calculate amplitude from power
def power_to_amplitude(image):
    return image.sqrt()

In [ ]:
# apply power -> amplitude scaling
s1_amplitude_asc = s1_power_asc.map(power_to_amplitude)

In [ ]:
# Visualize the results
Map = geemap.Map()

Map.setCenter(-73.8719, 4.512, 9);  # Bogota, Colombia

# Display the temporal means for various observations, compare them.
Map.addLayer(s1_power_asc.mean(), {"bands":"VV", "min": 0, "max": 0.3}, 'Power image');
Map.addLayer(s1_amplitude_asc.mean(), {"bands":"VV", "min": 0, "max": 0.75}, 'Amplitude image');

Map.addLayerControl()

Map

### dB Scale

The dB scale is calculated by multiplying 10 times the Log10 of the power scale values. This scale brightens the pixels, allowing for better differentiation among very dark pixels. When identifying water on the landscape, this is often a good scale to use; the water pixels generally remain very dark, while the terrestrial pixels are even brighter. Because this is in a log scale, it is not appropriate for all types of statistical analyses.

In [ ]:
# function to convert power to dB units
def power_to_db(image):
    return ee.Image.constant(10).multiply(image.log10())

In [ ]:
# apply power -> db scaling
s1_db_asc = s1_power_asc.map(power_to_db)

In [ ]:
# Visualize the results
Map = geemap.Map()

Map.setCenter(-73.8719, 4.512, 9);  # Bogota, Colombia

Map.addLayer(s1_power_asc.mean(), {"bands":"VV", "min": 0, "max": 0.3}, 'Power image');
Map.addLayer(s1_amplitude_asc.mean(), {"bands":"VV", "min": 0, "max": 0.75}, 'Amplitude image');
Map.addLayer(s1_db_asc.mean(), {"bands":"VV", "min": -25, "max": 0}, 'dB image');


Map.addLayerControl()

Map

## Radiometric Terrain Correction (RTC)

The data stored on Earth Engine comes without terrain correction, however, this is an important step in processing SAR for a variety of applications. When RTC is performed, foreshortened areas are corrected based on the DEM. Areas impacted by layover or shadow, however, do not actually have data returns to correct.

[Vollrath et al. (2020)](https://doi.org/10.3390/rs12111867) created a slope correction algorithm to be used with Earth Engine, here we will implement the methods and apply to SAR image for exploration.

In [ ]:
def slope_correction(
    image
):
    """This function applies the slope correction on a Sentinel-1 image.
    Function based on https:# doi.org/10.3390/rs12111867.
    Adapted from https:# github.com/ESA-PhiLab/radiometric-slope-correction/blob/master/notebooks/1%20-%20Generate%20Data.ipynb

    args:
        image (ee.Image): Sentinel-1 to perform correction on
        elevation (ee.Image): Input DEM to calculate slope corrections from
        model (str, optional): physical reference model to be applied. Options are 'volume' or 'surface'.
            default = volume
        buffer (int, optional): buffer in meters for layover/shadow mask. If zero then no buffer will be applied. default = 0
        scale (int, optional): reduction scale to process satellite heading compared to ground. Increasing will reduce
            chance of OOM errors but reduce local scale correction accuracy. default = 1000

    returns:
        ee.Image: slope corrected SAR imagery with look and local incidence angle bands

    raises:
        NotImplementedError: when keyword model is not of 'volume' or 'surface'
    """

    def _volumetric_model_SCF(theta_iRad, alpha_rRad):
        """Closure funnction for calculation of volumetric model SCF

        args:
            theta_iRad (ee.Image): incidence angle in radians
            alpha_rRad (ee.Image): slope steepness in range

        returns:
            ee.Image
        """

        # model
        nominator = (ninetyRad.subtract(theta_iRad).add(alpha_rRad)).tan()
        denominator = (ninetyRad.subtract(theta_iRad)).tan()
        return nominator.divide(denominator)

    def _surface_model_SCF(theta_iRad, alpha_rRad, alpha_azRad):
        """Closure funnction for calculation of direct model SCF

        args:
            theta_iRad (ee.Image): incidence angle in radians
            alpha_rRad (ee.Image): slope steepness in range
            alpha_azRad (ee.Image): slope steepness in azimuth

        returns:
            ee.Image
        """

        # model
        nominator = (ninetyRad.subtract(theta_iRad)).cos()
        denominator = alpha_azRad.cos().multiply(
            (ninetyRad.subtract(theta_iRad).add(alpha_rRad)).cos()
        )

        return nominator.divide(denominator)

    def _erode(image, distance):
        """Closure function to buffer raster values

        args:
            image (ee.Image): image that should be buffered
            distance (int): distance of buffer in meters

        returns:
            ee.Image
        """

        d = (
            image.Not()
            .unmask(1)
            .fastDistanceTransform(10)
            .sqrt()
            .multiply(ee.Image.pixelArea().sqrt())
        )

        return image.updateMask(d.gt(distance))

    def _masking(alpha_rRad, theta_iRad, buffer):
        """Closure function for masking of layover and shadow

        args:
            alpha_rRad (ee.Image): slope steepness in range
            theta_iRad (ee.Image): incidence angle in radians
            buffer (int): buffer in meters

        returns:
            ee.Image
        """
        # layover, where slope > radar viewing angle
        layover = alpha_rRad.lt(theta_iRad).rename("layover")

        # shadow
        shadow = alpha_rRad.gt(
            ee.Image.constant(-1).multiply(ninetyRad.subtract(theta_iRad))
        ).rename("shadow")

        # add buffer to layover and shadow
        if buffer > 0:
            layover = _erode(layover, buffer)
            shadow = _erode(shadow, buffer)

        # combine layover and shadow
        no_data_mask = layover.And(shadow).rename("no_data_mask")

        return no_data_mask

    # get the image geometry and projection
    geom = image.geometry(scale)
    proj = image.select(1).projection()
    angle_band = image.select("angle")

    # image to convert angle to radians
    to_radians = ee.Image.constant((math.pi / 180))
    # create a 90 degree image in radians
    ninetyRad = ee.Image.constant(90).multiply(to_radians)

    # calculate the look direction
    heading = (
        ee.Terrain.aspect(image.select("angle"))
        .reduceRegion(ee.Reducer.mean(), geom, scale)
        .get("aspect")
    )

    # the numbering follows the article chapters
    # 2.1.1 Radar geometry
    theta_iRad = image.select("angle").multiply(to_radians)
    phi_iRad = ee.Image.constant(heading).multiply(to_radians)

    # 2.1.2 Terrain geometry
    alpha_sRad = (
        ee.Terrain.slope(elevation)
        .select("slope")
        .multiply(to_radians)
        .setDefaultProjection(proj)
    )

    phi_sRad = (
        ee.Terrain.aspect(elevation)
        .select("aspect")
        .multiply(to_radians)
        .setDefaultProjection(proj)
    )

    # 2.1.3 Model geometry
    # reduce to 3 angle
    phi_rRad = phi_iRad.subtract(phi_sRad)

    # slope steepness in range (eq. 2)
    alpha_rRad = (alpha_sRad.tan().multiply(phi_rRad.cos())).atan()

    # slope steepness in azimuth (eq 3)
    alpha_azRad = (alpha_sRad.tan().multiply(phi_rRad.sin())).atan()

    # local incidence angle (eq. 4)
    theta_liaRad = (
        alpha_azRad.cos().multiply((theta_iRad.subtract(alpha_rRad)).cos())
    ).acos()
    theta_liaDeg = theta_liaRad.multiply(180 / math.pi)

    # 2.2
    # Gamma_nought
    gamma0 = image.divide(theta_iRad.cos())

    if model == "volume":
        scf = _volumetric_model_SCF(theta_iRad, alpha_rRad)

    elif model == "surface":
        scf = _surface_model_SCF(theta_iRad, alpha_rRad, alpha_azRad)

    else:
        raise NotImplementedError(
            f"Defined model, {model}, has not been implemented. Options are 'volume' or 'surface'"
        )

    # apply model for Gamm0_f
    gamma0_flat = gamma0.divide(scf)

    # calculate layover and shadow mask
    masks = _masking(alpha_rRad, theta_iRad, buffer)

    return (
        gamma0_flat.updateMask(masks)
        .addBands(angle_band)
        .addBands(theta_liaDeg.rename("local_inc_angle"))
    )

# define variables used within the slope_correction function
elevation = ee.Image("NASA/NASADEM_HGT/001").select("elevation")
model="volume"
buffer=100 # buffer areas of terrain shadow, in meters
scale=1000 # processing scale for reductions

In [ ]:
# apply slope correction to power data
power_rtc_asc = s1_power_asc.map(slope_correction)
# convert data from power to dB
db_rtc_asc = power_rtc_asc.map(power_to_db)

In [ ]:
# Visualize the results
Map = geemap.Map()

# Map.setCenter(-73.8719, 4.512, 9);  # Bogota, Colombia
Map.centerObject(s1_db_asc.first(),10)

Map.addLayer(s1_db_asc.mean(), {"bands":"VV", "min": -25, "max": 0}, 'dB image');
Map.addLayer(db_rtc_asc.mean(), {"bands":"VV", "min": -25, "max": 0}, 'dB RTC image');

Map.addLayerControl()

Map

## Speckle Filtering

Another important preprocessing step for SAR imagery is speckle filtering. In most cases, the patch of ground illuminated by the SAR transmitter will not be homogeneous. Instead it will be comprised of many different types of individual scatterers. The scatterers may interfere with each other either strengthening the return or weakening it. This creates a grainy (salt & pepper) appearance in SAR imagery. This a result of the nature of SAR and, thus, occurs in all SAR scenes. Speckle filtering attempts to mitigate the natural speckle in SAR imagery but there is a trade-off between reducing speckle and maintaining edges within the imagery. Because of this many different speckle filter algorithms have been developed. 

Here we will apply a compuationally efficient speckle filter algorithm, the Gamma Map algorithm  ([Lopes et al., 1990](https://doi.org/10.1109/36.62623)).

In [ ]:
# define a function to apply the gamma map speckle filter algorithm
def gamma_map(image):
    """Gamma Map speckle filtering algorithm.
    Algorithm adapted from https://groups.google.com/g/google-earth-engine-developers/c/a9W0Nlrhoq0/m/tnGMC45jAgAJ.

    args:
        img (ee.Image): Earth engine image object. Expects that imagery is a SAR image in power scale

    returns:
        ee.Image: filtered SAR power image using the Gamma Map algorithm
    """

    img_bands = image.bandNames()

    # Square kernel, window should be odd (typically 3, 5 or 7)
    weights = ee.List.repeat(ee.List.repeat(1, window), window)
    midPt = (window // 2) + 1 if (window % 2) != 0 else window // 2

    # ~~(window/2) does integer division in JavaScript
    kernel = ee.Kernel.fixed(window, window, weights, midPt, midPt, False)

    # Get mean and variance
    mean = image.reduceNeighborhood(ee.Reducer.mean(), kernel)
    variance = image.reduceNeighborhood(ee.Reducer.variance(), kernel)

    # "Pure speckle" threshold
    ci = variance.sqrt().divide(mean)  # square root of inverse of enl

    # If ci <= cu, the kernel lies in a "pure speckle" area -> return simple mean
    cu = 1.0 / math.sqrt(enl)

    # If cu < ci < cmax the kernel lies in the low textured speckle area -> return the filtered value
    cmax = math.sqrt(2.0) * cu

    alpha = ee.Image(1.0 + cu * cu).divide(ci.multiply(ci).subtract(cu * cu))
    b = alpha.subtract(enl + 1.0)
    d = (
        mean.multiply(mean)
        .multiply(b)
        .multiply(b)
        .add(alpha.multiply(mean).multiply(image).multiply(4.0 * enl))
    )
    f = b.multiply(mean).add(d.sqrt()).divide(alpha.multiply(2.0))

    caster = ee.Dictionary.fromLists(
        img_bands, ee.List.repeat("float", img_bands.length())
    )
    img1 = (
        mean.updateMask(ci.lte(cu))
        .rename(img_bands)
        .cast(caster)
    )
    img2 = (
        f.updateMask(ci.gt(cu)).updateMask(ci.lt(cmax))
        .rename(img_bands)
        .cast(caster)
    )
    img3 = image.updateMask(ci.gte(cmax)).rename(img_bands).cast(caster)

    # If ci > cmax do not filter at all (i.e. we don't do anything, other then masking)
    output = (
        ee.ImageCollection([img1, img2, img3])
        .reduce(ee.Reducer.firstNonNull())
        .rename(img_bands)
        .clip(image.geometry(1e3))
    )

    # Compose a 3 band image with the mean filtered "pure speckle", the "low textured" filtered and the unfiltered portions
    return output
    

window=7 # filtering window size
enl=4.9 # equivalent number of looks, for S1 enl ≈ 5

In [ ]:
# apply speckle filter to RTC imagery
power_filtered_asc = power_rtc_asc.map(gamma_map)
# convert filtered data from power to dB
db_filtered_asc = power_filtered_asc.map(power_to_db)

In [ ]:
# mosaic image and reproject for visualization
proj = ee.Projection("EPSG:4326").atScale(30)
db_filtered_asc_mean = db_filtered_asc.mean().reproject(proj)


In [ ]:
# Visualize the results
Map = geemap.Map()

Map.setCenter(-73.8719, 4.512, 9);  # Bogota, Colombia

Map.addLayer(s1_db_asc.mean(), {"bands":"VV", "min": -25, "max": 0}, 'dB image');
Map.addLayer(db_rtc_asc.mean(), {"bands":"VV", "min": -25, "max": 0}, 'dB RTC image');
Map.addLayer(db_filtered_asc_mean, {"bands":"VV", "min": -25, "max": 0}, 'dB RTC,filtered image');


Map.addLayerControl()

Map

Note: There are other speckle algorithms implemented in Earth Engine such as the [Lee Sigma](https://doi.org/10.1109/TGRS.2008.2002881) and [Refined Lee](https://doi.org/10.1109/36.789635) algorithms.

## Change detection

A simple and informative approach to change detection is the calculation of the log difference between two RTC datasets from different dates:

$ log10(\frac{obs_{t}}{obs_{t-n}})$

using this approach it is easy to identify areas where change occurred, as well as the direction of the change. Negative values indicate a decrease in radar backscatter over time, while positive values indicate an increase in backscatter.

We will apply this approach to identify areas of deforestation:

In [ ]:
# get an image collection of SAR power data for descending orbit
s1_power = (
    ee.ImageCollection("COPERNICUS/S1_GRD_FLOAT")
    # Filter to get images with VV and VH dual polarization.
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
    # Filter to get images collected in interferometric wide swath mode.
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    # filter orbit pass
    .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
)

In [ ]:
# filter for June 2019
# apply slope correction and speckle filter
before = s1_power.filterDate('2019-06-01', '2019-07-01').map(slope_correction).map(gamma_map)
# convert to amplitude scale for change detection
before_amp = before.map(power_to_amplitude).mean()

In [ ]:
# filter for June 2020
# apply slope correction and speckle filter
after = s1_power.filterDate('2020-06-01', '2020-07-01').map(slope_correction).map(gamma_map)
# convert to amplitude scale for change detection
after_amp = after.map(power_to_amplitude).mean()

In [ ]:
# apply log ratio change detection
change = after_amp.divide(before_amp).log10()

In [ ]:
# Visualize the results
Map = geemap.Map()

Map.setCenter(-57.70219, -13.266281, 9);  # Amazon Forest

Map.addLayer(before_amp.reproject(proj), {"bands":"VH", "min": 0, "max": 0.75}, 'before image');
Map.addLayer(after_amp.reproject(proj), {"bands":"VH", "min": 0, "max": 0.75}, 'after image');
Map.addLayer(change.reproject(proj), {"bands":"VH", "min": -0.5, "max": 0.5, "palette":cmaps.get_palette("PiYG")}, 'change image');


Map.addLayerControl()

Map

In [ ]:
# create a map of deforestation based on change magnitude from threshold 
deforested = change.lt(-0.2)

In [ ]:
# Visualize the results
Map = geemap.Map()

Map.setCenter(-57.70219, -13.266281, 9);  # Amazon Forest

Map.addLayer(after_amp.reproject(proj), {"bands":"VH", "min": 0, "max": 0.75}, 'after image');
Map.addLayer(deforested.selfMask().reproject(proj), {"bands":"VH", "min": 0, "max": 1, "palette":"red"}, 'Deforestation');


Map.addLayerControl()

Map